## 142 SAT [Enhancement] Märk ut saknade taggar för framkomlighetsbedömning

### version 3.3 

* this [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/142_3%20SAT%20%5BEnhancement%5D%20M%C3%A4rk%20ut%20saknade%20taggar%20f%C3%B6r%20framkomlighetsbed%C3%B6mning.ipynb)
  * Latest maps [Image Quality Control](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_image_qc_latest.html) / [Global Audit](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_ALL_latest.html) / [Task Map](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_split_todo_latest.html)
 

---- 
* Issue [142 SAT Enhancement Märk ut saknade taggar för framkomlighetsbedömning](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142)
 

* Try to get better quality of trail data in OSM

---- 
* Version 3.3 creates three maps

       
    
  
  


In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-09-13 12:14:24


In [2]:
# =================== SAT: ALL-IN-ONE (WD + OSM + AUDIT + POPUP + MAP SAVE) ===================
from __future__ import annotations

# --- Stdlib
import os
import re
import time
import json
import shutil
from pathlib import Path
from urllib.parse import quote, unquote
from string import Template
from datetime import datetime
import html as _html
import html

# --- Third-party
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import folium
from folium import GeoJson, GeoJsonTooltip, GeoJsonPopup

# =================== Projekt/Output =========================================
PROJECT_NAME = "SAT_ALL_IN_ONE_142_3"
OUTPUT_DIR   = "./output"  # ändra vid behov
stamp        = pd.Timestamp.now().strftime("%Y%m%d_%H%M")

WD_ENDPOINT  = "https://query.wikidata.org/sparql"
OVERPASS_URL = "https://overpass-api.de/api/interpreter"
UA           = {"User-Agent": "SAT-etapper/1.0 (salgo60 tools)"}

WIKI_KEY_URLS = {
    "trail_visibility": "https://wiki.openstreetmap.org/wiki/Key:trail_visibility",
    "sac_scale":        "https://wiki.openstreetmap.org/wiki/Key:sac_scale",
    "mtb:scale":        "https://wiki.openstreetmap.org/wiki/Key:mtb:scale",
    "mtb:scale:imba":   "https://wiki.openstreetmap.org/wiki/Key:mtb:scale:imba",
    "surface":          "https://wiki.openstreetmap.org/wiki/Key:surface",
    "smoothness":       "https://wiki.openstreetmap.org/wiki/Key:smoothness",
    "tracktype":        "https://wiki.openstreetmap.org/wiki/Key:tracktype",
    "highway":          "https://wiki.openstreetmap.org/wiki/Key:highway",
    "foot":             "https://wiki.openstreetmap.org/wiki/Key:foot",
    "bicycle":          "https://wiki.openstreetmap.org/wiki/Key:bicycle",
    "width":            "https://wiki.openstreetmap.org/wiki/Key:width",
    "incline":          "https://wiki.openstreetmap.org/wiki/Key:incline",
    "lit":              "https://wiki.openstreetmap.org/wiki/Key:lit",
    "step_count":       "https://wiki.openstreetmap.org/wiki/Key:step_count",
    "ford":             "https://wiki.openstreetmap.org/wiki/Key:ford",
    "bridge":           "https://wiki.openstreetmap.org/wiki/Key:bridge",
    "tunnel":           "https://wiki.openstreetmap.org/wiki/Key:tunnel",
    "wheelchair":       "https://wiki.openstreetmap.org/wiki/Key:wheelchair",
    "ramp":             "https://wiki.openstreetmap.org/wiki/Key:ramp",
    "osmc:symbol":      "https://wiki.openstreetmap.org/wiki/Key:osmc:symbol",
}

# =================== Konstanter/Färger ======================================
AUDIT_COLORS = {
    "missing_surface":   "#ff8800",
    "missing_access":    "#cc0000",
    "missing_hike":      "#7e57c2",
    "inconsistent":      "#ffd54f",
    "missing_stepcount": "#8d6e63",
    "tracks_compacted":  "#2b8cbe",
    "missing_mtbscale":  "#00acc1",
    "missing_tracktype": "#ef6c00",
    "scenic":            "#43a047",
}

# =================== Hjälpare: OSM-länkar ===================================
def _mk_osm_links(osmid) -> str:
    """Bygger klicklänkar till OSM-view, iD-editor och historiksidor."""
    try:
        osmid = int(osmid)
        view  = f"https://www.openstreetmap.org/way/{osmid}"
        edit  = f"https://www.openstreetmap.org/edit?editor=id&way={osmid}"
        hist  = f"https://pewu.github.io/osm-history/#/way/{osmid}"
        hist2 = f"https://osm.mapki.com/history/way/{osmid}"
        return (
            f'<a href="{view}" target="_blank">OSM way {osmid}</a> · '
            f'<a href="{edit}" target="_blank">Öppna i iD</a> · '
            f'<a href="{hist}" target="_blank">Historik</a> · '
            f'<a href="{hist2}" target="_blank">metadata historik</a>'
        )
    except Exception:
        return ""

def _mk_mapillary_link(row) -> str:
    """HTML-länk till Mapillary-foto om tagg finns, annars karta på centroid."""
    mkey = None
    try:
        mkey = row.get("mapillary")
    except Exception:
        pass

    if isinstance(mkey, str) and mkey.strip():
        key = mkey.split(";")[0].strip()
        url = f"https://www.mapillary.com/app/?pKey={quote(key)}&focus=photo"
        return f'<a href="{url}" target="_blank">Mapillary foto</a>'

    try:
        geom = row.get("geometry")
        if geom is not None:
            c = geom.centroid
            lat, lon = float(c.y), float(c.x)
            url = f"https://www.mapillary.com/app/?z=18&lat={lat:.6f}&lng={lon:.6f}&focus=map"
            return f'<a href="{url}" target="_blank">Mapillary här</a>'
    except Exception:
        pass
    return ""
def _format_wikimap_coords(lat: float, lon: float, prec: int = 6) -> tuple[str, str]:
    """
    Wikimap likes zero-padded degrees like:
      lat=059.412525  (3 digits)
      lon=0018.556753 (4 digits)
    Keep sign if S/W, pad absolute value, 6 decimals by default.
    """
    def fmt(v: float, width: int) -> str:
        sign = "-" if v < 0 else ""
        return f"{sign}{abs(v):0{width}.{prec}f}"
    return fmt(lat, 3), fmt(lon, 4)

def _mk_wikimap_link(row, zoom: int = 16, cluster: bool = False, wp: bool = False) -> str:
    """HTML-link to Wikimap at the feature centroid with padded coords."""
    try:
        geom = row.get("geometry")
        if geom is None:
            return ""
        c = geom.centroid
        lat, lon = float(c.y), float(c.x)
        s_lat, s_lon = _format_wikimap_coords(lat, lon, prec=6)
        url = (
            "https://wikimap.toolforge.org/"
            f"?wp={'true' if wp else 'false'}"
            f"&cluster={'true' if cluster else 'false'}"
            f"&zoom={int(zoom)}&lat={s_lat}&lon={s_lon}"
        )
        return f'<a href="{url}" target="_blank">Wikimap här</a>'
    except Exception:
        return ""
# =================== Steps-only karta =======================================
def make_steps_only_map(etapp_gdfs: dict[str, gpd.GeoDataFrame],
                        OUTPUT_DIR: str = OUTPUT_DIR,
                        PROJECT_NAME: str = PROJECT_NAME,
                        stamp: str = stamp) -> str:
    # 1) Samla allt i en DF
    dfs = []
    for etapp_label, gdf in etapp_gdfs.items():
        if gdf is None or gdf.empty:
            continue
        gf = gdf.copy()
        gf["name"] = gf.get("name", pd.Series([None] * len(gf))).fillna(etapp_label)
        dfs.append(gf)

    if not dfs:
        print("Inga segment att visa (steps).")
        return ""

    df_all = pd.concat(dfs, ignore_index=True).copy()

    # säkerställ kolumner vi använder
    required = ["osmid","geometry","name","highway","surface","tracktype",
                "foot","bicycle","sac_scale","trail_visibility","smoothness","width",
                "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit",
                "step_count","ford","bridge","tunnel","wheelchair","ramp","source","mapillary"]
    for c in required:
        if c not in df_all.columns:
            df_all[c] = None

    # 2) Filtrera ut enbart trappor
    H = df_all["highway"].astype(str).str.lower().replace({"none": ""})
    steps = df_all.loc[H == "steps"].copy()
    if steps.empty:
        print("Hittade inga trappor (highway=steps).")
        return ""

    # 3) Popup-byggare (återanvänder existerande badges, länkar m.m.)
    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = " · ".join(filter(None, [
            _mk_osm_links(row.get("osmid")),
            _mk_wikimap_link(row),
            _mk_mapillary_link(row),
        ]))
        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    steps["popup_html"] = steps.apply(_build_popup_html_from_row, axis=1)

    # 4) Masker: med/utan step_count
    s = steps["step_count"].fillna("").astype(str).str.strip().str.lower()
    has_count = (s != "") & (s != "nan")
    no_count  = ~has_count

    # 5) Karta + lager
    gdf_steps = gpd.GeoDataFrame(steps, geometry="geometry", crs="EPSG:4326")
    minx, miny, maxx, maxy = gdf_steps.total_bounds
    m = folium.Map(location=[(miny + maxy) / 2, (minx + maxx) / 2],
                   zoom_start=12, control_scale=True, tiles="OpenStreetMap")

    # ljus bakgrund av alla etapp-geometrier (valfritt)
    try:
        bg_all = pd.concat([gdf.loc[:, ["geometry"]] for gdf in etapp_gdfs.values() if gdf is not None and not gdf.empty],
                           ignore_index=True)
        if not bg_all.empty:
            GeoJson(bg_all, name="Etapper (bakgrund)",
                    style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.35}).add_to(m)
    except Exception:
        pass

    def _add_steps_layer(gdf_sel: gpd.GeoDataFrame, name: str, color_hex: str):
        if gdf_sel.empty:
            return
        gj = GeoJson(
            data=gdf_sel[["geometry", "popup_html", "name", "step_count"]],
            name=name,
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=["name", "step_count"],
                                   aliases=["Section", "step_count"],
                                   sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=name)
        gj.add_to(fg)
        fg.add_to(m)

    COLOR_HAS_COUNT = "#1f6feb"                # blå
    COLOR_NO_COUNT  = AUDIT_COLORS["missing_stepcount"]  # brun från din audit-palett

    _add_steps_layer(gdf_steps.loc[has_count].copy(), "Trappor ✔ med step_count", COLOR_HAS_COUNT)
    _add_steps_layer(gdf_steps.loc[no_count].copy(),  "Trappor ⚠ saknar step_count", COLOR_NO_COUNT)

    # enkel legend
    legend_html = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Trappor (highway=steps)</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_HAS_COUNT};margin-right:6px;"></span>✔ med step_count</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_NO_COUNT};margin-right:6px;"></span>⚠ saknar step_count</div>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))
    folium.LayerControl(collapsed=False).add_to(m)

    # fit bounds & spara
    m.fit_bounds([[gdf_steps.total_bounds[1], gdf_steps.total_bounds[0]],
                  [gdf_steps.total_bounds[3], gdf_steps.total_bounds[2]]])

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_steps_only_{stamp}.html")
    add_about_box(m, issue_number=142, map_name="Steps only")
    save_with_latest(m, html_path)
    print("Steps-only karta sparad till:", html_path)
    return html_path
    

# =================== Hjälpare: Commons-bilder ===============================
def _is_http_url(s: str) -> bool:
    return isinstance(s, str) and s.lower().startswith(("http://", "https://"))

def normalize_commons_title(image_tag: str) -> str | None:
    """Normaliserar in till 'File:Title.ext' (stöder /wiki/File:... och upload-URL:er)."""
    if not isinstance(image_tag, str) or not image_tag.strip():
        return None
    s = image_tag.strip()

    m = re.search(r'(?:^|/wiki/)(File:[^#?]+)$', s, flags=re.I)
    if m:
        raw = m.group(1)
        return "File:" + unquote(raw.split(":", 1)[1])

    if "upload.wikimedia.org" in s:
        last = s.split("/")[-1]
        last = re.sub(r'^\d+px-', '', last)   # ta bort tumnagel-prefix
        return "File:" + unquote(last)

    if not _is_http_url(s):
        if s.lower().startswith("file:"):
            return "File:" + unquote(s.split(":", 1)[1])
        return "File:" + unquote(s)
    return None

def commons_thumb_url(title: str, width: int = 600) -> str:
    if not isinstance(title, str) or not title:
        return ""
    if not title.lower().startswith("file:"):
        title = "File:" + title
    decoded = "File:" + unquote(title.split(":", 1)[1])
    safe = quote(decoded, safe="/:")
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{safe}?width={width}"

def commons_filepage_url(title: str) -> str:
    if not isinstance(title, str) or not title:
        return ""
    if not title.lower().startswith("file:"):
        title = "File:" + title
    decoded = unquote(title.split(":", 1)[1]).replace(" ", "_")
    safe = quote("File:" + decoded, safe="/:")
    return f"https://commons.wikimedia.org/wiki/{safe}"

def get_commons_entity_json_for_title(title: str, timeout=10) -> dict | None:
    if not isinstance(title, str) or not title:
        return None
    if not title.lower().startswith("file:"):
        title = "File:" + title
    decoded = "File:" + unquote(title.split(":", 1)[1])
    url = f"https://commons.wikimedia.org/wiki/Special:EntityData/{quote(decoded, safe='/:')}.json"
    try:
        r = requests.get(url, headers=UA, timeout=timeout)
        if r.status_code == 200:
            return r.json()
    except Exception:
        pass
    return None

def commons_has_p10689(title: str, osmid: int | str) -> tuple[bool, str | None]:
    data = get_commons_entity_json_for_title(title, timeout=6)
    if not data or "entities" not in data:
        return (False, None)
    for mid, ent in data["entities"].items():
        for claim in ent.get("statements", {}).get("P10689", []) or []:
            val = claim.get("mainsnak", {}).get("datavalue", {}).get("value")
            if str(val) == str(osmid):
                return (True, mid)
    return (False, None)

def make_image_html(image_value: str) -> str:
    if not image_value:
        return ""
    title = normalize_commons_title(image_value)
    if title:
        thumb = commons_thumb_url(title, width=600)
        link  = commons_filepage_url(title)
        return (f'<a href="{html.escape(link)}" target="_blank">'
                f'<img src="{html.escape(thumb)}" style="max-width:100%;height:auto;border-radius:8px"/></a>')
    if _is_http_url(image_value):
        return f'<img src="{html.escape(image_value)}" style="max-width:100%;height:auto;border-radius:8px"/>'
    return ""
def first_thumb_url(image_value: str | None, width: int = 320) -> str:
    """
    Returnerar en tumnagel-URL för Commons 'File:*' eller original-URL om det är http(s).
    Inget Mapillary-fallback. Tom sträng om inget går att använda.
    """
    if not image_value:
        return ""
    title = normalize_commons_title(image_value)
    if title:
        return commons_thumb_url(title, width=width)
    if _is_http_url(str(image_value)):
        return str(image_value)
    return ""

# =================== Badge/Popup-hjälpare ===================================
def _fmt(v: str | None) -> str:
    return "" if v is None else str(v).replace("_", " ").replace("-", " ").strip()

def _tag(tags: dict, *keys, default=None):
    for k in keys:
        if k in tags and str(tags[k]).strip():
            return tags[k]
    return default

def _badge(text: str, kind: str = "default") -> str:
    palette = {
        "primary":"#1f6feb","success":"#238636","warning":"#9e6a03",
        "danger":"#b62324","muted":"#57606a","default":"#6e7781",
    }
    color = palette.get(kind, palette["default"])
    return (f'<span style="display:inline-block;padding:2px 8px;border-radius:999px;'
            f'font-size:12px;line-height:18px;background:{color};color:white;'
            f'margin:0 6px 6px 0;white-space:nowrap;">{html.escape(text)}</span>')

def _sac_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    name = {
        "hiking":"hiking",
        "mountain_hiking":"mountain hiking",
        "demanding_mountain_hiking":"demanding mountain hiking",
        "alpine_hiking":"alpine hiking",
        "demanding_alpine_hiking":"demanding alpine hiking",
        "difficult_alpine_hiking":"difficult alpine hiking",
    }.get(v, v)
    kind = "success" if v=="hiking" else ("warning" if v in {"mountain_hiking","demanding_mountain_hiking"} else "danger")
    return _badge(f"sac_scale: {_fmt(name)}", kind)

def _trail_visibility_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    kind = "success" if v in {"excellent","good"} else ("warning" if v=="intermediate" else ("danger" if v in {"bad","none"} else "default"))
    return _badge(f"visibility: {_fmt(v)}", kind)

def _surface_badge(val: str | None) -> str:     return _badge(f"surface: {_fmt(val)}","muted") if val else ""
def _width_badge(val: str | None) -> str:       return _badge(f"width: {_fmt(val)}","muted")   if val else ""
def _smoothness_badge(val: str | None) -> str:  return _badge(f"smoothness: {_fmt(val)}","muted") if val else ""
def _osmc_symbol_badge(val: str | None) -> str: return _badge(f"osmc:symbol: {_fmt(val)}","primary") if val else ""
def _sat_color_hint(val: str | None) -> str:
    if not val: return ""
    return _badge("SAT blå/gul","primary") if ("blue" in val.lower() and "yellow" in val.lower()) else ""
def _foot_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    return _badge(f"foot: {_fmt(v)}", "success" if v in {"designated","yes"} else "default")
def _highway_badge(val: str | None) -> str:     return _badge(f"highway: {_fmt(val)}","default") if val else ""
def _incline_badge(val: str | None) -> str:     return _badge(f"incline: {val}","muted") if val else ""
def _lit_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    return _badge("lit","success") if v=="yes" else (_badge("unlit","muted") if v=="no" else _badge(f"lit: {v}","muted"))
def _steps_badge(step_count: str | None, highway: str | None) -> str:
    return _badge(f"steps: {step_count}","default") if (str(highway).lower()=="steps" and step_count) else ""
def _crossings_badge(ford: str | None, bridge: str | None, tunnel: str | None) -> str:
    chips = []
    if str(bridge).lower()=="yes": chips.append(_badge("bridge","primary"))
    if str(ford).lower()=="yes":   chips.append(_badge("ford","warning"))
    if str(tunnel).lower()=="yes": chips.append(_badge("tunnel","default"))
    return "".join(chips)
def _wheelchair_badge(val: str | None) -> str:
    if not val: return ""
    v = str(val).lower()
    return _badge(f"wheelchair: {v}", "success" if v=="yes" else ("warning" if v=="limited" else "muted"))
def _source_badge(val: str | None) -> str:      return _badge(f"source: {_fmt(val)}", "primary") if val else ""

def _collect_images_from_tags(tags: dict, max_n: int = 6) -> list[str]:
    imgs = []
    base = _tag(tags, "image")
    if base: imgs.append(base)
    for i in range(max_n):
        k = f"image:{i}"
        if k in tags and str(tags[k]).strip():
            imgs.append(tags[k])
    seen, uniq = set(), []
    for it in imgs:
        if it not in seen:
            uniq.append(it); seen.add(it)
    return uniq[:max_n]

def _render_images(img_values: list[str]) -> str:
    if not img_values: return ""
    cards = []
    for v in img_values:
        img_html = make_image_html(v)
        if img_html:
            cards.append(f'<div style="flex:1 1 240px;max-width:100%;padding:4px;">{img_html}</div>')
    if not cards: return ""
    return ('<div style="display:flex;flex-wrap:wrap;gap:4px;margin-top:8px;'
            'align-items:flex-start;justify-content:flex-start;">' + "".join(cards) + "</div>")

def _collect_source_subtags(tags: dict) -> list[str]:
    return [f"{k}={tags[k]}" for k in sorted(tags) if k.startswith("source:") and str(tags[k]).strip()]

# =================== Popup: Vandringsfokus ==================================
def build_hiking_popup_html(tags: dict, title: str | None = None, osmid=None, extra_html: str = "") -> str:
    highway    = _tag(tags, "highway")
    foot       = _tag(tags, "foot")
    sac        = _tag(tags, "sac_scale")
    visibility = _tag(tags, "trail_visibility")
    surface    = _tag(tags, "surface")
    tracktype  = _tag(tags, "tracktype")
    width      = _tag(tags, "width")
    smooth     = _tag(tags, "smoothness")
    mtb_scale  = _tag(tags, "mtb:scale", "mtb:scale:imba")
    bicycle    = _tag(tags, "bicycle")
    osmc_sym   = _tag(tags, "osmc:symbol")

    incline    = _tag(tags, "incline")
    lit        = _tag(tags, "lit")
    step_count = _tag(tags, "step_count")
    ford       = _tag(tags, "ford")
    bridge     = _tag(tags, "bridge")
    tunnel     = _tag(tags, "tunnel")
    wheelchair = _tag(tags, "wheelchair")
    ramp       = _tag(tags, "ramp")
    source_tag = _tag(tags, "source")

    badges_primary = "".join([_highway_badge(highway), _foot_badge(foot), _sac_badge(sac)])
    badges_nav     = _trail_visibility_badge(visibility)
    badges_surface = "".join(filter(None, [
        _surface_badge(surface),
        _badge(f"tracktype: {_fmt(tracktype)}","muted") if tracktype else ""
    ]))
    badges_cycling = "".join(filter(None, [
        _width_badge(width), _smoothness_badge(smooth),
        _badge(f"mtb:scale: {_fmt(mtb_scale)}","default") if mtb_scale else "",
        _badge(f"bicycle: {_fmt(bicycle)}","default") if bicycle else "",
        _osmc_symbol_badge(osmc_sym), _sat_color_hint(osmc_sym),
    ]))
    badges_cross = "".join(filter(None, [
        _crossings_badge(ford, bridge, tunnel),
        _steps_badge(step_count, highway),
        _incline_badge(incline),
        _lit_badge(lit),
        _wheelchair_badge(wheelchair),
    ]))

    src_sub       = _collect_source_subtags(tags)
    badges_source = " ".join(_badge(s, "primary") for s in src_sub) or _source_badge(source_tag)

    warnings = []
    if (highway in {"path","footway"}) and (str(foot).lower() in {"designated","yes"}):
        miss = []
        if not sac:        miss.append("sac_scale")
        if not visibility: miss.append("trail_visibility")
        if miss:
            warnings.append(_badge("⚠ saknar " + ", ".join(miss), "danger"))
    if tracktype in {"grade1","grade2","grade3","grade4","grade5"} and highway != "track":
        warnings.append(_badge("⚠ tracktype utan highway=track", "warning"))

    img_values = _collect_images_from_tags(tags)
    img_html   = _render_images(img_values)

    if osmid is not None and img_values:
        t = normalize_commons_title(img_values[0])
        if t:
            ok, _ = commons_has_p10689(t, osmid)
            if ok:
                warnings.append(_badge("✔ Bild kopplad via P10689", "success"))

    title_html = f'<div style="font-weight:700;font-size:18px;margin:0 0 6px 0;">{html.escape(title)}</div>' if title else ""
    desc_bits  = []
    if sac:        desc_bits.append(f"sac_scale: {_fmt(sac)}")
    if visibility: desc_bits.append(f"visibility: {_fmt(visibility)}")
    if surface:    desc_bits.append(f"surface: {_fmt(surface)}")
    desc_html = ('<div style="color:#57606a;font-size:13px;margin:2px 0 8px 0;">'
                 "Vandringsprofil: " + ", ".join(map(html.escape, desc_bits)) + "</div>") if desc_bits else ""
    warning_html = "".join(warnings)

    def section(emoji, text, content):
        if not content: return ""
        return (f'<div style="margin:6px 0 2px 0;font-weight:600;font-size:13px;">{emoji} {text}</div>'
                f'<div style="margin:4px 0 8px 0;">{content}</div>')

    present_keys = []
    for k in ["highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype",
              "smoothness","width","mtb:scale","mtb:scale:imba","osmc:symbol",
              "incline","lit","step_count","ford","bridge","tunnel","wheelchair","ramp"]:
        v = _tag(tags, k)
        if v is not None and str(v).strip():
            present_keys.append(k)

    doc_links = []
    for k in present_keys:
        wl = _wiki_link_for_key(k)
        if wl: doc_links.append(wl)
        doc_links.append(_taginfo_link_for_key(k))

    html_sections = "".join([
        section("🥾","Grundtaggar", badges_primary),
        section("🧭","Synlighet & orientering", badges_nav),
        section("🪵","Underlag", badges_surface),
        section("🚲","Cykel / MTB", badges_cycling),
        section("🌉","Passager & hinder", badges_cross),
        section("🗂️","Källa", badges_source),
        section("📚","Dokumentation", "".join(doc_links)),
        section("🖼️","Bilder", img_html),
    ])

    outer = (
        '<div style="max-width:520px">'
        '<div style="border-radius:12px;padding:10px;background:#ffffff;'
        'box-shadow:0 1px 6px rgba(0,0,0,.08);font-family:system-ui,-apple-system,'
        'Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;font-size:14px;color:#24292f;">'
        f'{title_html}'
        '<div style="display:flex;align-items:center;gap:8px;margin:4px 0 8px 0;">'
        '<div style="font-weight:700;">🥾 Vandring</div>'
        f'{warning_html}'
        '</div>'
        f'{desc_html}{html_sections}{extra_html}'
        "</div></div>"
    )
    return outer

# =================== Legend ==================================================
def add_audit_legend(m: folium.Map, include_tracks=False):
    lines = [
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_surface"]};margin-right:6px;"></span>saknar <code>surface=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_access"]};margin-right:6px;"></span>saknar <code>foot=*</code> / <code>bicycle=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_hike"]};margin-right:6px;"></span>saknar <code>sac_scale</code> / <code>trail_visibility</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["inconsistent"]};margin-right:6px;"></span><code>tracktype=*</code> utan <code>highway=track</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_stepcount"]};margin-right:6px;"></span><code>highway=steps</code> utan <code>step_count=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_mtbscale"]};margin-right:6px;"></span>saknar <code>mtb:scale</code> / <code>mtb:scale:imba</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["missing_tracktype"]};margin-right:6px;"></span><code>highway=track</code> utan <code>tracktype=*</code></div>',
      f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["scenic"]};margin-right:6px;"></span><code>scenic=yes</code> (tematiskt lager)</div>',
    ]
    if include_tracks:
        lines.append(f'<div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS["tracks_compacted"]};margin-right:6px;"></span>Tracks (<code>surface=compacted</code>)</div>')
    html_legend = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Datakvalitet (globala lager)</div>
      {''.join(lines)}
    </div>
    """
    m.get_root().html.add_child(folium.Element(html_legend))

# =================== Wikidata: hämta SAT-etapper ============================
def fetch_sat_sections_from_wikidata(trail_qid: str = "Q131318799") -> pd.DataFrame:
    query = f"""
    SELECT ?section ?sectionLabel ?osmr ?osmw_p10689 ?osmw_p11693 WHERE {{
      VALUES ?trail {{ wd:{trail_qid} }}
      {{ ?section wdt:P361+ ?trail . }} UNION {{ ?trail wdt:P527 ?section . }}
      FILTER(?section != ?trail)
      OPTIONAL {{ ?section wdt:P402 ?osmr }}
      OPTIONAL {{ ?section wdt:P10689 ?osmw_p10689 }}
      OPTIONAL {{ ?section wdt:P11693 ?osmw_p11693 }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    ORDER BY ?sectionLabel
    """
    headers = {"Accept": "application/sparql-results+json", **UA}
    r = requests.get(WD_ENDPOINT, params={"query": query}, headers=headers, timeout=60)
    r.raise_for_status()
    data = r.json()
    rows = []
    for b in data["results"]["bindings"]:
        qid   = b["section"]["value"].split("/")[-1]
        label = b.get("sectionLabel", {}).get("value", qid)
        osmr  = b.get("osmr", {}).get("value")
        osmw1 = b.get("osmw_p10689", {}).get("value")
        osmw2 = b.get("osmw_p11693", {}).get("value")
        osmw  = osmw1 or osmw2
        rows.append({
            "section_qid": qid,
            "sectionLabel": label,
            "osmr": pd.to_numeric(osmr, errors="coerce"),
            "osmw": pd.to_numeric(osmw, errors="coerce"),
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df["osmr"] = df["osmr"].astype("Int64")
        df["osmw"] = df["osmw"].astype("Int64")
    return df

# =================== Overpass-hjälpare ======================================
def _overpass(query: str, retries: int = 3, pause: float = 2.0) -> dict:
    for i in range(retries):
        try:
            resp = requests.post(OVERPASS_URL, data={"data": query}, headers=UA, timeout=180)
            if resp.status_code == 429 and i < retries-1:
                time.sleep(pause * (i+1)); continue
            resp.raise_for_status()
            return resp.json()
        except Exception:
            if i == retries-1:
                raise
            time.sleep(pause * (i+1))
    return {"elements": []}

def fetch_ways_for_relation(rel_id: int) -> list[dict]:
    q = f"""
    [out:json][timeout:180];
    relation({rel_id});
    way(r);
    out tags geom;
    """
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]

def fetch_way(way_id: int) -> list[dict]:
    q = f"""
    [out:json][timeout:180];
    way({way_id});
    out tags geom;
    """
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]

# =================== Konvertering: Overpass -> GeoDataFrame =================
_CORE_TAGS = [
    "name","highway","surface","tracktype","foot","bicycle",
    "sac_scale","trail_visibility","smoothness","width",
    "mtb:scale","mtb:scale:imba","osmc:symbol","image",
    "incline","lit","step_count","ford","bridge","tunnel","wheelchair","ramp",
    "source","mapillary","scenic",
]

def _way_to_feature(el: dict) -> dict | None:
    if "geometry" not in el: return None
    coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
    if len(coords) < 2: return None
    tags = el.get("tags", {}) or {}
    props = {"osmid": el["id"]}
    for k in _CORE_TAGS:
        props[k] = tags.get(k)
    return {"properties": props, "geometry": LineString(coords)}

def elements_to_gdf(elements: list[dict]) -> gpd.GeoDataFrame:
    feats = []
    for el in elements:
        f = _way_to_feature(el)
        if f: feats.append(f)
    if not feats:
        return gpd.GeoDataFrame(columns=["osmid",*_CORE_TAGS,"geometry"], geometry="geometry", crs="EPSG:4326")
    df  = pd.DataFrame([f["properties"] for f in feats])
    gdf = gpd.GeoDataFrame(df, geometry=[f["geometry"] for f in feats], crs="EPSG:4326")
    for k in ["osmid",*_CORE_TAGS]:
        if k not in gdf.columns: gdf[k] = None
    return gdf

# =================== Bygg seg_gdf_4326 per etapp ============================
def build_sat_seg_gdfs(trail_qid: str = "Q131318799", prefer_relation: bool = True) -> dict[str, gpd.GeoDataFrame]:
    meta = fetch_sat_sections_from_wikidata(trail_qid)
    if meta.empty:
        print("Hittade inga etapper i Wikidata för", trail_qid)
        return {}

    out: dict[str, gpd.GeoDataFrame] = {}
    for _, row in meta.iterrows():
        label  = str(row["sectionLabel"]).strip() or row["section_qid"]
        rel_id = row["osmr"] if pd.notna(row["osmr"]) else None
        way_id = row["osmw"] if pd.notna(row["osmw"]) else None

        elements: list[dict] = []
        if prefer_relation and rel_id:
            try:
                elements = fetch_ways_for_relation(int(rel_id))
            except Exception as e:
                print(f"[{label}] Fel vid hämtning av relation {rel_id}: {e}")

        if (not elements) and way_id:
            try:
                elements = fetch_way(int(way_id))
            except Exception as e:
                print(f"[{label}] Fel vid hämtning av way {way_id}: {e}")

        gdf = elements_to_gdf(elements)
        if "name" not in gdf.columns:
            gdf["name"] = None
        gdf["name"] = gdf["name"].fillna(label)
        out[label] = gdf

    return out

# =================== Wiki / Taginfo länkar ==================================
def _wiki_link_for_key(key: str) -> str:
    url = WIKI_KEY_URLS.get(key)
    if not url: return ""
    return (f'<a href="{html.escape(url)}" target="_blank" '
            f'style="display:inline-block;padding:2px 8px;border-radius:999px;'
            f'font-size:12px;line-height:18px;background:#1f6feb;color:white;'
            f'margin:0 6px 6px 0;white-space:nowrap;text-decoration:none;">'
            f'Wiki: {html.escape(key)}</a>')

def _taginfo_link_for_key(key: str) -> str:
    url = f"https://taginfo.openstreetmap.org/keys/{quote(key, safe='')}"
    return (f'<a href="{html.escape(url)}" target="_blank" '
            f'style="display:inline-block;padding:2px 8px;border-radius:999px;'
            f'font-size:12px;line-height:18px;background:#6e7781;color:white;'
            f'margin:0 6px 6px 0;white-space:nowrap;text-decoration:none;">'
            f'Taginfo: {html.escape(key)}</a>')

# =================== EN karta med GLOBALE lager per kategori ================
def make_all_in_one_map_global(etapp_gdfs: dict[str, gpd.GeoDataFrame], mode: str = "foot",
                               OUTPUT_DIR: str = OUTPUT_DIR, PROJECT_NAME: str = PROJECT_NAME,
                               stamp: str = stamp, include_tracks_compacted: bool = True) -> str:

    bounds = []
    acc = {
        "missing_surface":   [], "missing_access":   [], "missing_hike":    [],
        "inconsistent":      [], "missing_stepcount":[], "background":      [],
        "all_segments":      [], "missing_mtbscale": [], "missing_tracktype":[],
        "scenic":            [],
    }
    if include_tracks_compacted:
        acc["tracks_compacted"] = []

    for etapp_label, seg_gdf_4326 in etapp_gdfs.items():
        if seg_gdf_4326 is None or seg_gdf_4326.empty:
            continue

        minx, miny, maxx, maxy = seg_gdf_4326.to_crs(epsg=4326).total_bounds
        bounds.append((minx, miny, maxx, maxy))

        sg = seg_gdf_4326.copy()
        for col in ["highway","surface","foot","bicycle","sac_scale","trail_visibility",
                    "tracktype","smoothness","width","mtb:scale","mtb:scale:imba",
                    "osmc:symbol","incline","lit","step_count","ford","bridge","tunnel","wheelchair","ramp","source"]:
            if col not in sg.columns:
                sg.loc[:, col] = None
            sg.loc[:, col] = sg[col].astype(str).str.lower().replace({"none": ""})

        missing_surface = sg.loc[(sg["surface"] == "") | (sg["surface"].isna())].copy()
        if mode == "foot":
            miss_access = sg.loc[(sg["highway"].isin(["path","footway"])) & (sg["foot"].isin(["","nan"]) | sg["foot"].isna())].copy()
        else:
            miss_access = sg.loc[(sg["bicycle"].isin(["","nan"]) | sg["bicycle"].isna())].copy()

        miss_hike = sg.loc[
            sg["highway"].isin(["path","footway"]) &
            ((sg["sac_scale"].isin(["","nan"]) | sg["sac_scale"].isna()) |
             (sg["trail_visibility"].isin(["","nan"]) | sg["trail_visibility"].isna()))
        ].copy()

        inconsistent = sg.loc[
            (sg["tracktype"].isin(["grade1","grade2","grade3","grade4","grade5"])) &
            (sg["highway"] != "track")
        ].copy()

        missing_stepcount = sg.loc[(sg["highway"] == "steps") & (sg["step_count"].isin(["","nan"]) | sg["step_count"].isna())].copy()
        missing_mtbscale  = sg.loc[sg["highway"].isin(["path","track"]) & (sg["mtb:scale"].isin(["","nan"]) | sg["mtb:scale"].isna()) &
                                   (sg["mtb:scale:imba"].isin(["","nan"]) | sg["mtb:scale:imba"].isna())].copy()
        missing_tracktype = sg.loc[(sg["highway"] == "track") & (sg["tracktype"].isin(["","nan"]) | sg["tracktype"].isna())].copy()
        has_scenic        = sg.loc[(sg["scenic"] == "yes")].copy()

        # name-fyllning
        for df in [missing_surface, miss_access, miss_hike, inconsistent, missing_stepcount, missing_mtbscale]:
            if "name" not in df.columns:
                df.loc[:, "name"] = etapp_label
            else:
                df.loc[:, "name"] = df["name"].fillna(etapp_label)

        acc["missing_surface"].append(missing_surface)
        acc["missing_access"].append(miss_access)
        acc["missing_hike"].append(miss_hike)
        acc["inconsistent"].append(inconsistent)
        acc["missing_stepcount"].append(missing_stepcount)
        acc["background"].append(seg_gdf_4326.loc[:, ["geometry"]].copy())
        acc["missing_mtbscale"].append(missing_mtbscale)
        acc["missing_tracktype"].append(missing_tracktype)
        acc["scenic"].append(has_scenic)

        if include_tracks_compacted:
            tracks_compacted = sg.loc[(sg["highway"] == "track") & (sg["surface"] == "compacted")].copy()
            if "name" not in tracks_compacted.columns:
                tracks_compacted.loc[:, "name"] = etapp_label
            else:
                tracks_compacted.loc[:, "name"] = tracks_compacted["name"].fillna(etapp_label)
            acc["tracks_compacted"].append(tracks_compacted)

        seg_full = seg_gdf_4326.copy()
        if "name" not in seg_full.columns:
            seg_full["name"] = etapp_label
        else:
            seg_full["name"] = seg_full["name"].fillna(etapp_label)
        acc["all_segments"].append(seg_full)

    # Init-karta
    if not bounds:
        m = folium.Map(location=[59.3293, 18.0686], zoom_start=9, control_scale=True, tiles="OpenStreetMap")
    else:
        minx = min(b[0] for b in bounds); miny = min(b[1] for b in bounds)
        maxx = max(b[2] for b in bounds); maxy = max(b[3] for b in bounds)
        m = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")

    # Popup-hjälpare för rader
    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = " · ".join(filter(None, [
                _mk_osm_links(row.get("osmid")),
                _mk_wikimap_link(row),          
                _mk_mapillary_link(row),
            ]))

        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    # Global lager-hjälpare
    def _make_global_layer(df_list, layer_name, color_hex, reason_text=None):
        dfs = [d for d in df_list if (d is not None and not d.empty)]
        if not dfs: return None
        df = pd.concat(dfs, ignore_index=True).copy()
        if reason_text: df.loc[:, "audit_reason"] = reason_text
        df.loc[:, "popup_html"] = df.apply(_build_popup_html_from_row, axis=1)

        fields = ["geometry", "popup_html", "highway", "surface", "tracktype"]
        tooltip_fields  = ["highway","surface","tracktype"]
        tooltip_aliases = ["highway","surface","tracktype"]
        if reason_text:
            fields.append("audit_reason"); tooltip_fields.append("audit_reason"); tooltip_aliases.append("varför i detta lager")

        gdf = gpd.GeoDataFrame(df[fields], geometry="geometry", crs="EPSG:4326")
        gj = GeoJson(
            data=gdf, name=layer_name,
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases, sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=layer_name)
        gj.add_to(fg)
        return fg

    # Globala lager
    L1 = _make_global_layer(acc["missing_surface"],   "Saknar surface", AUDIT_COLORS["missing_surface"])
    L2 = _make_global_layer(acc["missing_access"],    f"Saknar {'foot' if mode=='foot' else 'bicycle'}", AUDIT_COLORS["missing_access"])
    L3 = _make_global_layer(acc["missing_hike"],      "Saknar sac/visibility", AUDIT_COLORS["missing_hike"],
                            "highway=path/footway och saknar sac_scale eller trail_visibility")
    L4 = _make_global_layer(acc["inconsistent"],      "Inkonsekvent tracktype", AUDIT_COLORS["inconsistent"])
    L5 = _make_global_layer(acc["missing_stepcount"], "Saknar step_count (trappor)", AUDIT_COLORS["missing_stepcount"])
    L6 = _make_global_layer(acc["missing_mtbscale"],  "Saknar mtb:scale/mtb:scale:imba", AUDIT_COLORS["missing_mtbscale"],
                            "highway=path/track saknar mtb:scale och mtb:scale:imba")
    L7 = _make_global_layer(acc["missing_tracktype"], "Saknar tracktype (på highway=track)", AUDIT_COLORS["missing_tracktype"],
                            "highway=track men saknar tracktype")
    L8 = _make_global_layer(acc["scenic"],            "Scenic (scenic=yes)", AUDIT_COLORS["scenic"], "scenic=yes")

    for L in [L1, L2, L3, L4, L5, L6, L7, L8]:
        if L is not None: L.add_to(m)

    if include_tracks_compacted and "tracks_compacted" in acc:
        LT = _make_global_layer(acc["tracks_compacted"], "Tracks (surface=compacted)", AUDIT_COLORS["tracks_compacted"])
        if LT is not None: LT.add_to(m)

    if acc["background"]:
        bg = pd.concat(acc["background"], ignore_index=True)
        GeoJson(bg, name="Etapper (bakgrund)",
                style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.6}).add_to(m)
    # Alla segment (popup) + hover-bild (endast image, ingen text/Mapillary-fallback)
    if acc["all_segments"]:
        df_all = pd.concat(acc["all_segments"], ignore_index=True).copy()
    
        # säkerställ kolumner
        for col in ["highway","foot","bicycle","sac_scale","trail_visibility","surface",
                    "tracktype","smoothness","width","mtb:scale","mtb:scale:imba",
                    "osmc:symbol","image","incline","lit","step_count","ford","bridge",
                    "tunnel","wheelchair","ramp","source","name","osmid","geometry","scenic"]:
            if col not in df_all.columns:
                df_all[col] = None
    
        # popup html som tidigare
        df_all["popup_html"] = df_all.apply(_build_popup_html_from_row, axis=1)
    
        # hover: bygg en ren bild-url från 'image' (ingen fallback, ingen text)
        df_all["hover_img"] = df_all["image"].apply(lambda v: first_thumb_url(v, width=320))
    
        gdf_all = gpd.GeoDataFrame(
            df_all[["geometry","popup_html","hover_img"]],
            geometry="geometry", crs="EPSG:4326"
        )
    
        # Spara lagrets variabelnamn för att kunna koppla JS-händelser
        gj_all = GeoJson(
            data=gdf_all,
            name="Alla segment (popup)",
            style_function=lambda _f: {"weight": 3, "color": "#555", "opacity": 0.9},
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        gj_all.add_to(m)
    
        # Litet hover-fönster som följer musen
        m.get_root().html.add_child(folium.Element(f"""
    <div id="sat-hover-preview" style="position: fixed; display: none; pointer-events: none; z-index: 10000;
      border: 1px solid #ccc; background: #fff; padding: 3px; border-radius: 6px; box-shadow: 0 1px 6px rgba(0,0,0,.2);">
      <img id="sat-hover-img" style="max-width: 220px; max-height: 160px; display: block;" />
    </div>
    <script>
    (function(){{
      var layer = {gj_all.get_name()};
      var box = document.getElementById('sat-hover-preview');
      var img = document.getElementById('sat-hover-img');
    
      function move(ev){{
        var e = ev.originalEvent || ev;
        var x = e.clientX || 0;
        var y = e.clientY || 0;
        box.style.left = (x + 16) + 'px';
        box.style.top  = (y + 16) + 'px';
      }}
      function show(url, ev){{
        if(!url) return;
        if (img.getAttribute('src') !== url) img.setAttribute('src', url);
        box.style.display = 'block';
        move(ev);
      }}
      function hide(){{
        box.style.display = 'none';
        img.removeAttribute('src');
      }}
    
      // Bind mouse events per feature — bara om vi har hover_img
      layer.eachLayer(function(l){{
        var props = l && l.feature && l.feature.properties;
        if (props && props.hover_img){{
          l.on('mouseover', function(ev){{ show(props.hover_img, ev); }});
          l.on('mousemove', move);
          l.on('mouseout', hide);
        }}
      }});
    }})();
    </script>
        """))
    
    
        add_audit_legend(m, include_tracks=include_tracks_compacted)
        folium.LayerControl(collapsed=False).add_to(m)
    
        # === Way index (ID-lista) ===============================================
        if acc.get("all_segments"):
            df_idx = pd.concat(acc["all_segments"], ignore_index=True).copy()
            needed_cols = ["osmid","name","highway","surface","foot","bicycle",
                           "sac_scale","trail_visibility","tracktype","step_count",
                           "mtb:scale","mtb:scale:imba"]
            for c in needed_cols:
                if c not in df_idx.columns: df_idx[c] = None
    
            def _is_missing(v):
                s = str(v).strip().lower() if v is not None else ""
                return s in {"", "nan", "none"}
    
            def _todo(row):
                todo = []
                hwy = str(row.get("highway") or "").lower()
                if _is_missing(row.get("surface")): todo.append("add surface")
                if mode == "foot":
                    if hwy in {"path","footway"} and _is_missing(row.get("foot")): todo.append("add foot")
                else:
                    if _is_missing(row.get("bicycle")): todo.append("add bicycle")
                if hwy in {"path","footway"} and (_is_missing(row.get("sac_scale")) or _is_missing(row.get("trail_visibility"))):
                    todo.append("add sac/visibility")
                tt = str(row.get("tracktype") or "").lower()
                if tt in {"grade1","grade2","grade3","grade4","grade5"} and hwy != "track":
                    todo.append("fix tracktype vs highway")
                if hwy == "steps" and _is_missing(row.get("step_count")):
                    todo.append("add step_count")
                if hwy in {"path","track"} and _is_missing(row.get("mtb:scale")) and _is_missing(row.get("mtb:scale:imba")):
                    todo.append("add mtb:scale/imba")
                if hwy == "track" and _is_missing(row.get("tracktype")):
                    todo.append("add tracktype")
                return ", ".join(todo)
    
            df_idx = df_idx[df_idx["osmid"].notnull()].copy()
            if not df_idx.empty:
                df_idx["osmid"] = df_idx["osmid"].astype(int)
                df_idx["what_to_do"] = df_idx.apply(_todo, axis=1)
                df_idx["name"] = df_idx["name"].fillna("Okänd etapp")
                df_idx.sort_values(["name","osmid"], inplace=True)
    
                def _idx_table_html(df):
                    rows, current_section = [], None
                    for _, r in df.iterrows():
                        sec = str(r["name"])
                        if sec != current_section:
                            current_section = sec
                            rows.append(f"<tr class='section'><td colspan='4' style='padding:10px 8px;font-weight:700;background:#f8fafc;border-top:1px solid #eee;'>{html.escape(sec)}</td></tr>")
                        osm_id = int(r["osmid"])
                        view  = f"https://www.openstreetmap.org/way/{osm_id}"
                        edit  = f"https://www.openstreetmap.org/edit?editor=id&way={osm_id}"
                        todo  = html.escape(r.get("what_to_do") or "")
                        rows.append(
                            "<tr>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'><a href='{view}' target='_blank'>{osm_id}</a></td>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'><a href='{edit}' target='_blank'>iD</a></td>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'>{html.escape(sec)}</td>"
                            f"<td style='padding:8px;border-bottom:1px solid #f1f5f9;'>{todo}</td>"
                            "</tr>"
                        )
                    return "\n".join(rows)
    
                table_html = f"""
                <div id="way-index" style="margin: 24px auto; max-width: 1100px;
                     font-family: system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, Helvetica, Arial, sans-serif;">
                  <h2 style="margin:0 0 8px 0;">Way Index</h2>
                  <div style="font-size:13px; color:#57606a; margin-bottom:8px">
                    ID klickable | ID editor | Section | what to do
                  </div>
                  <input id="wayFilter" placeholder="Filter (id, section, todo...)"
                         style="padding:8px 10px; width:100%; box-sizing:border-box; margin-bottom:8px;
                                border-radius:8px; border:1px solid #e5e7eb;">
                  <div style="overflow:auto; max-height: 50vh; border:1px solid #e5e7eb; border-radius:10px;">
                    <table style="width:100%; border-collapse:collapse; font-size:14px;">
                      <thead style="position: sticky; top: 0; background:#fafafa; z-index: 1;">
                        <tr>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">ID</th>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">Edit</th>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">Section</th>
                          <th style="text-align:left; padding:8px; border-bottom:1px solid #eee;">What to do</th>
                        </tr>
                      </thead>
                      <tbody id="wayIndexBody">
                        {_idx_table_html(df_idx)}
                      </tbody>
                    </table>
                  </div>
                </div>
                <script>
                  (function(){{
                    var filter = document.getElementById('wayFilter');
                    if(!filter) return;
                    filter.addEventListener('input', function() {{
                      var q = this.value.toLowerCase();
                      var rows = document.querySelectorAll('#wayIndexBody tr');
                      rows.forEach(function(r){{
                        if (r.classList.contains('section')) {{
                          r.style.display = (q=='' ? '' : 'none');
                          return;
                        }}
                        var text = r.innerText.toLowerCase();
                        r.style.display = text.indexOf(q) !== -1 ? '' : 'none';
                      }});
                    }});
                  }})();
                </script>
                """
                jump_btn = """
                <a href="#way-index" title="Hoppa till ID-listan"
                   style="position: fixed; bottom: 18px; right: 18px; z-index: 9999; background: #111827; color: white;
                          padding: 10px 12px; border-radius: 999px; text-decoration: none;
                          box-shadow: 0 2px 8px rgba(0,0,0,0.25); font-size: 13px;">
                  ID-lista ⤵
                </a>
                """
                m.get_root().html.add_child(folium.Element(jump_btn))
                m.get_root().html.add_child(folium.Element(table_html))
    
        # Fit bounds
        if bounds:
            m.fit_bounds([[min(b[1] for b in bounds), min(b[0] for b in bounds)],
                          [max(b[3] for b in bounds), max(b[2] for b in bounds)]])
    
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_ALL_{stamp}.html")
        add_about_box(m, issue_number=142, map_name="Todo")
        save_with_latest(m, html_path)
        print("Samlad karta (globala lager) sparad till:", html_path)
    return html_path

# =================== Split TODO-karta =======================================
def make_split_todo_map(etapp_gdfs: dict[str, gpd.GeoDataFrame], mode: str = "foot",
                        OUTPUT_DIR: str = OUTPUT_DIR, PROJECT_NAME: str = PROJECT_NAME, stamp: str = stamp) -> str:

    dfs = []
    for etapp_label, gdf in etapp_gdfs.items():
        if gdf is None or gdf.empty: continue
        gf = gdf.copy()
        gf["name"] = gf.get("name", pd.Series([None]*len(gf))).fillna(etapp_label)
        dfs.append(gf)
    if not dfs:
        print("Inga segment att visa."); return ""

    df_all = pd.concat(dfs, ignore_index=True).copy()
    required = ["osmid","geometry","name","highway","surface","foot","bicycle",
                "sac_scale","trail_visibility","tracktype","step_count",
                "mtb:scale","mtb:scale:imba","smoothness","width","osmc:symbol",
                "image","incline","lit","ford","bridge","tunnel","wheelchair","ramp","source","mapillary"]
    for c in required:
        if c not in df_all.columns: df_all[c] = None

    def _norm(s): return s.astype(str).str.lower().replace({"none": ""})
    for col in ["highway","surface","foot","bicycle","sac_scale","trail_visibility","tracktype","step_count","mtb:scale","mtb:scale:imba"]:
        df_all[col] = _norm(df_all[col])

    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = " · ".join(filter(None, [
                _mk_osm_links(row.get("osmid")),
                _mk_wikimap_link(row),          
                _mk_mapillary_link(row),
            ]))

        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    df_all["popup_html"] = df_all.apply(_build_popup_html_from_row, axis=1)

    def _is_missing(series: pd.Series) -> pd.Series:
        s = series.fillna("").astype(str).str.strip().str.lower()
        return (s == "") | (s == "nan")

    H = df_all["highway"]
    m_surface   = _is_missing(df_all["surface"])
    if mode == "foot":
        m_access = (H.isin(["path","footway"])) & _is_missing(df_all["foot"]); access_label = "add foot"
    else:
        m_access = _is_missing(df_all["bicycle"]); access_label = "add bicycle"
    m_hike      = H.isin(["path","footway"]) & (_is_missing(df_all["sac_scale"]) | _is_missing(df_all["trail_visibility"]))
    m_incon_tt  = df_all["tracktype"].isin(["grade1","grade2","grade3","grade4","grade5"]) & (H != "track")
    m_step      = (H == "steps") & _is_missing(df_all["step_count"])
    m_mtb       = H.isin(["path","track"]) & _is_missing(df_all["mtb:scale"]) & _is_missing(df_all["mtb:scale:imba"])
    m_tt_on_trk = (H == "track") & _is_missing(df_all["tracktype"])

    todo_specs = [
        ("add surface",                 m_surface,   AUDIT_COLORS["missing_surface"]),
        (access_label,                  m_access,    AUDIT_COLORS["missing_access"]),
        ("add sac/visibility",          m_hike,      AUDIT_COLORS["missing_hike"]),
        ("fix tracktype vs highway",    m_incon_tt,  AUDIT_COLORS["inconsistent"]),
        ("add step_count",              m_step,      AUDIT_COLORS["missing_stepcount"]),
        ("add mtb:scale/imba",          m_mtb,       AUDIT_COLORS["missing_mtbscale"]),
        ("add tracktype",               m_tt_on_trk, AUDIT_COLORS["missing_tracktype"]),
    ]

    gdf_all = gpd.GeoDataFrame(df_all, geometry="geometry", crs="EPSG:4326")
    if gdf_all.empty:
        m2 = folium.Map(location=[59.3293, 18.0686], zoom_start=9, control_scale=True, tiles="OpenStreetMap")
    else:
        minx, miny, maxx, maxy = gdf_all.total_bounds
        m2 = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")

    GeoJson(data=gdf_all[["geometry"]], name="Etapper (bakgrund)",
            style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.5}).add_to(m2)

    def _add_todo_layer(mask: pd.Series, layer_name: str, color_hex: str):
        gsel = gdf_all.loc[mask].copy()
        if gsel.empty: return
        gsel["tooltip_section"] = gsel["name"].fillna("")
        gsel["tooltip_id"]      = gsel["osmid"].fillna("").astype(str)
        gj = GeoJson(
            data=gsel[["geometry","popup_html","tooltip_section","tooltip_id"]],
            name=f"TODO: {layer_name}",
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=["tooltip_id","tooltip_section"], aliases=["ID","Section"], sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=f"TODO: {layer_name}")
        gj.add_to(fg); fg.add_to(m2)
        m2.get_root().html.add_child(folium.Element(f"""
        <script>
          (function() {{
            window.__todoLayers = window.__todoLayers || [];
            window.__todoLayers.push({gj.get_name()});
          }})();
        </script>
        """))

    for lname, lmask, lcolor in todo_specs:
        _add_todo_layer(lmask, lname, lcolor)

    add_audit_legend(m2, include_tracks=False)
    folium.LayerControl(collapsed=False).add_to(m2)

    if not gdf_all.empty:
        m2.fit_bounds([[gdf_all.total_bounds[1], gdf_all.total_bounds[0]],
                       [gdf_all.total_bounds[3], gdf_all.total_bounds[2]]])

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path2 = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_split_todo_{stamp}.html")
    add_about_box(m2, issue_number=142, map_name="Audit layers")
    save_with_latest(m2, html_path2)
    print("Split-todo karta sparad till:", html_path2)
    return html_path2

# =================== Image QC-karta =========================================
def make_image_qc_map(etapp_gdfs: dict[str, gpd.GeoDataFrame],
                      OUTPUT_DIR: str = OUTPUT_DIR, PROJECT_NAME: str = PROJECT_NAME, stamp: str = stamp) -> str:

    dfs = []
    for etapp_label, gdf in etapp_gdfs.items():
        if gdf is None or gdf.empty: continue
        gf = gdf.copy()
        gf["name"] = gf.get("name", pd.Series([None]*len(gf))).fillna(etapp_label)
        dfs.append(gf)
    if not dfs:
        print("Inga segment att visa (image QC)."); return ""

    df_all = pd.concat(dfs, ignore_index=True).copy()
    required = ["osmid","geometry","name","image","highway","surface","tracktype",
                "foot","bicycle","sac_scale","trail_visibility","smoothness","width",
                "mtb:scale","mtb:scale:imba","osmc:symbol","incline","lit",
                "step_count","ford","bridge","tunnel","wheelchair","ramp","source","mapillary"]
    for c in required:
        if c not in df_all.columns: df_all[c] = None

    def _build_popup_html_from_row(row):
        tags = {k: row.get(k) for k in [
            "highway","foot","bicycle","sac_scale","trail_visibility","surface","tracktype","smoothness","width",
            "mtb:scale","mtb:scale:imba","osmc:symbol","image","incline","lit","step_count","ford","bridge",
            "tunnel","wheelchair","ramp","source","mapillary"
        ]}
        title = row.get("name") or f"Way {row.get('osmid')}"
        links = " · ".join(filter(None, [
                _mk_osm_links(row.get("osmid")),
                _mk_wikimap_link(row),          
                _mk_mapillary_link(row),
            ]))

        extra = f'<div style="margin-top:8px;">{links}</div>'
        return build_hiking_popup_html(tags, title=title, osmid=row.get("osmid"), extra_html=extra)

    s = df_all["image"].fillna("").astype(str).str.strip()
    df_all["has_image"] = s.ne("") & s.str.lower().ne("nan")

    def _check_p10689(row):
        try:
            if not row["has_image"] or pd.isna(row["osmid"]): return False
            title = normalize_commons_title(str(row["image"]))
            if not title: return False
            ok, _ = commons_has_p10689(title, int(row["osmid"]))
            return bool(ok)
        except Exception:
            return False

    df_all["has_p10689"] = df_all.apply(_check_p10689, axis=1)
    df_all["popup_html"] = df_all.apply(_build_popup_html_from_row, axis=1)
    gdf_all = gpd.GeoDataFrame(df_all, geometry="geometry", crs="EPSG:4326")

    if gdf_all.empty:
        m3 = folium.Map(location=[59.3293, 18.0686], zoom_start=9, control_scale=True, tiles="OpenStreetMap")
    else:
        minx, miny, maxx, maxy = gdf_all.total_bounds
        m3 = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=10, control_scale=True, tiles="OpenStreetMap")

    GeoJson(data=gdf_all[["geometry"]], name="Etapper (bakgrund)",
            style_function=lambda _f: {"weight": 3, "color": "#666", "opacity": 0.5}).add_to(m3)

    def _add_layer(mask: pd.Series, layer_name: str, color_hex: str):
        gsel = gdf_all.loc[mask].copy()
        if gsel.empty: return
        gj = GeoJson(
            data=gsel[["geometry","popup_html","name"]],
            name=layer_name,
            style_function=lambda _f, _c=color_hex: {"weight": 6, "color": _c},
            tooltip=GeoJsonTooltip(fields=["name"], aliases=["Section"], sticky=True),
            popup=GeoJsonPopup(fields=["popup_html"], aliases=[""], parse_html=True, labels=False, max_width=520)
        )
        fg = folium.FeatureGroup(name=layer_name)
        gj.add_to(fg); fg.add_to(m3)

    COLOR_HAS = "#27ae60"
    COLOR_NO  = "#e67e22"
    COLOR_P89 = "#1f6feb"

    _add_layer(gdf_all["has_image"],  "✔ Has image (P18/tag)",       COLOR_HAS)
    _add_layer(~gdf_all["has_image"], "✖ Missing image",             COLOR_NO)
    _add_layer(gdf_all["has_p10689"], "✔ Image linked via P10689",   COLOR_P89)

    legend_html = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Bildkvalitet</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_HAS};margin-right:6px;"></span>Har bild (P18/tag)</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_P89};margin-right:6px;"></span>Bild kopplad via P10689</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{COLOR_NO};margin-right:6px;"></span>Saknar bild</div>
    </div>
    """
    m3.get_root().html.add_child(folium.Element(legend_html))
    folium.LayerControl(collapsed=False).add_to(m3)

    if not gdf_all.empty:
        m3.fit_bounds([[gdf_all.total_bounds[1], gdf_all.total_bounds[0]],
                       [gdf_all.total_bounds[3], gdf_all.total_bounds[2]]])

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path3 = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_image_qc_{stamp}.html")
    add_about_box(m3, issue_number=142, map_name="Image Quality Control")
    save_with_latest(m3, html_path3)

    print("Image-QC karta sparad till:", html_path3)
    return html_path3

# =================== About-box (Notebook/Edge-safe) =========================
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    """
    Notebook/Edge-säker About-box med string.Template ($-placeholder).
    """
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
  @media (max-width: 640px) { .sat-about { font-size: 11px; min-width: 200px; max-width: 260px; } }
</style>

<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">$links_html</div>
  </div>
</div>

<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";

  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }

  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      var willCollapse = !box.classList.contains("sat-about-collapsed");
      setCollapsed(box, willCollapse);
    });

    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }

  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_snippet = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        collapsed_class=collapsed_class,
        issue_url=issue_url,
        issue_number=str(issue_number),
        map_name=_html.escape(map_name),
        created_date=_html.escape(created_date),
        links_html=links_html,
        map_dom_id=map_dom_id,
    )
    m.get_root().html.add_child(folium.Element(html_snippet))

# =================== Save helpers ===========================================
def _to_latest_name(filename: str | Path) -> Path:
    p = Path(filename)
    stem = p.stem
    new_stem = re.sub(r'_(?:20\d{6})(?:_\d{4})?$', '', stem)
    return p.with_name(new_stem + "_latest.html")

def save_with_latest(map_object, filename: str | Path):
    map_object.save(str(filename))
    latest_path = _to_latest_name(filename)
    shutil.copyfile(filename, latest_path)
    print(f"Saved: {filename}\nUpdated: {latest_path}")

# =================== Körning ================================================
etapp_gdfs = build_sat_seg_gdfs(trail_qid="Q131318799", prefer_relation=True)

html_path = make_all_in_one_map_global(
    etapp_gdfs=etapp_gdfs,
    mode="foot",                   # eller "bike"
    OUTPUT_DIR=OUTPUT_DIR,
    PROJECT_NAME=PROJECT_NAME,
    stamp=stamp,
    include_tracks_compacted=True
)
print("✅ All-in-one (globala lager) =>", html_path)

html_split = make_split_todo_map(
    etapp_gdfs=etapp_gdfs,
    mode="foot",                   # håll i sync med första kartan
    OUTPUT_DIR=OUTPUT_DIR,
    PROJECT_NAME=PROJECT_NAME,
    stamp=stamp
)
print("✅ Split TODO =>", html_split)

html_imgqc = make_image_qc_map(
    etapp_gdfs=etapp_gdfs,
    OUTPUT_DIR=OUTPUT_DIR,
    PROJECT_NAME=PROJECT_NAME,
    stamp=stamp
)
print("✅ Image QC =>", html_imgqc)

html_steps = make_steps_only_map(
    etapp_gdfs=etapp_gdfs,
    OUTPUT_DIR=OUTPUT_DIR,
    PROJECT_NAME=PROJECT_NAME,
    stamp=stamp
)
print("✅ Steps only =>", html_steps)



Saved: ./output/SAT_ALL_IN_ONE_142_3_ALL_20250913_1214.html
Updated: output/SAT_ALL_IN_ONE_142_3_ALL_latest.html
Samlad karta (globala lager) sparad till: ./output/SAT_ALL_IN_ONE_142_3_ALL_20250913_1214.html
✅ All-in-one (globala lager) => ./output/SAT_ALL_IN_ONE_142_3_ALL_20250913_1214.html
Saved: ./output/SAT_ALL_IN_ONE_142_3_split_todo_20250913_1214.html
Updated: output/SAT_ALL_IN_ONE_142_3_split_todo_latest.html
Split-todo karta sparad till: ./output/SAT_ALL_IN_ONE_142_3_split_todo_20250913_1214.html
✅ Split TODO => ./output/SAT_ALL_IN_ONE_142_3_split_todo_20250913_1214.html
Saved: ./output/SAT_ALL_IN_ONE_142_3_image_qc_20250913_1214.html
Updated: output/SAT_ALL_IN_ONE_142_3_image_qc_latest.html
Image-QC karta sparad till: ./output/SAT_ALL_IN_ONE_142_3_image_qc_20250913_1214.html
✅ Image QC => ./output/SAT_ALL_IN_ONE_142_3_image_qc_20250913_1214.html
Saved: ./output/SAT_ALL_IN_ONE_142_3_steps_only_20250913_1214.html
Updated: output/SAT_ALL_IN_ONE_142_3_steps_only_latest.html
Steps-

In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))


Date: 2025-09-13 12:26:06
Total time elapsed: 702.70 seconds
